## ETL: base de datos de peliculas

## 0.0 importamos las libreria

In [2]:
import pandas as pd
import ast
import numpy as np
pd.set_option('display.max_columns', None)

### 1.0 Extracción de los datos, se cargan los datasets (en formato csv) credits y movies_dataset 

In [3]:

df_credit= pd.read_csv(("../dataset/credits.csv"))
df_credit.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


### 1.1 Cargamos los datos 

In [4]:

df_movie = pd.read_csv("../dataset/movies_dataset.csv", low_memory=False)
df_movie.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

### Revisamos la informacion del dataset 

In [5]:
df_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

### 1.1 Creamos una función con el objetivo de expandir la lista de diccionarios que tenia el dataset credits, para eso usamos los Métodos clave

- **`ast.literal_eval()`**: Utilizado para convertir de manera segura cadenas que representan estructuras de datos Python en objetos nativos 
  
- **`explode()`**: Transforma listas en columnas en filas individuales, facilitando el análisis de datos que contienen listas de elementos relacionados.

- **`json_normalize()`**: Aplana estructuras de datos JSON anidadas en un DataFrame tabular


In [6]:
def expandircolumnas(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
        # Cambiamos con ast.literal_eval la forma que se interpretan los datos de str a tipo de dato py 
        df = df.explode(column)# Expandimos la columna
        
        col_df = pd.json_normalize(df[column]) #Normalizamos la columna 
        
        col_df = col_df.add_prefix(f'{column}') # Renombramos las columnas normalizadas a fin de evitar superposiciones
        # Concatenamos las columnas normalizadas con el DataFrame y borramos las columnas repetidas para evitar futuros conflictos
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df) 
    return df



### 1.1 Ponemos la funcion expandir columnas en marcha y desanidamos las columnas del dataset que vienen en formato de lista de diccionario

In [7]:
columns_to_expand = ['genres', 'production_companies', 'production_countries', 'spoken_languages'] #Columnas del dataset que debemos desanidar

df_movie = expandircolumnas(df_movie, columns_to_expand) 

### 1.2 Revisamos si tenemos valores nulos

In [8]:
df_movie.isna().sum() #sumamos la cantidad de valores nulos con los json desanidados


adult                                  0
belongs_to_collection             365398
budget                                 0
homepage                          291174
id                                     0
imdb_id                               65
original_language                     96
original_title                         0
overview                            2509
popularity                            17
poster_path                          580
release_date                         179
revenue                               21
runtime                              577
status                               165
tagline                           192313
title                                 21
video                                 21
vote_average                          21
vote_count                            21
genresid                            3201
genresname                          3201
production_companiesname           27098
production_companiesid             27098
production_count

In [9]:
df_credit.isna().sum()

cast    0
crew    0
id      0
dtype: int64

In [10]:
df_movie.duplicated().sum()

708

### 1.3 Cambiamos los valores nulos por '0' de las columnas revenue y budget

In [11]:
#df_movie["revenue"].isna().sum() #contamos la cantidad de valores nulos
df_movie["revenue"] = df_movie["revenue"].fillna(0)
df_movie["revenue"].isna().sum()

0

In [12]:

df_movie["budget"].isna().sum()#Revisamos la cantidad de valores nulos y comprobamos que no tenia ningun nulo

0

### 1.4 revisamos que release_date no tenga valores nulos y la cambiamos a datetime

In [13]:
df_movie["release_date"].isna().sum() #contamos la cantidad de valores nulos que tiene realease_date

179

In [14]:
df_movie = df_movie.dropna(subset=['release_date'])
df_movie["release_date"].isna().sum() #Eliminamos los nulos y revisamos la cantidad nuevamente

0

In [15]:
df_movie[(df_movie["release_date"] == '1')] #valor detectado que genera errores
df_movie['release_date'] = pd.to_datetime(df_movie['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')#cambiamos el formato de release a datetime 

### Un vez que tenemos el formato de realease_date en datetime y creamos la columna de realease_year para tener el año en que se realizó la película.

In [16]:
df_movie['release_date'] = pd.to_datetime(df_movie['release_date'])
df_movie['release_year'] = df_movie['release_date'].apply(lambda x: x.year)
df_movie['release_year'] = df_movie['release_date'].apply(lambda x: x.year if pd.notnull(x) else None).astype('Int64')
df_movie.info()


<class 'pandas.core.frame.DataFrame'>
Index: 419600 entries, 0 to 419778
Data columns (total 29 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   adult                           419600 non-null  object        
 1   belongs_to_collection           54372 non-null   object        
 2   budget                          419600 non-null  object        
 3   homepage                        128533 non-null  object        
 4   id                              419600 non-null  object        
 5   imdb_id                         419539 non-null  object        
 6   original_language               419504 non-null  object        
 7   original_title                  419600 non-null  object        
 8   overview                        417105 non-null  object        
 9   popularity                      419592 non-null  object        
 10  poster_path                     419081 non-null  object      

### 2.0 Eliminamos las columnas que no vamos a usar en el dataset

In [17]:

columnas_a_eliminar = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage',]
df_movie = df_movie.drop(columns=columnas_a_eliminar)



In [18]:
cantidad_columnas = len(df_movie.columns)

# Mostramos la cantidad de columnas
print(f'La cantidad de columnas es: {cantidad_columnas}')

La cantidad de columnas es: 23


### 1.6 revisamos la cantidad de valores duplicados en el dataset y eliminamos los valores duplicados

In [19]:
cantidad_duplicados = df_movie.duplicated().sum()

print("Cantidad de filas duplicadas:", cantidad_duplicados)

Cantidad de filas duplicadas: 708


In [20]:
df_movie.drop_duplicates(inplace=True)


In [21]:
cantidad_duplicados = df_movie.duplicated().sum()

print("Cantidad de filas duplicadas:", cantidad_duplicados)

Cantidad de filas duplicadas: 0


### 1.7 encontramos registros mal cargados y se los elimina del dataset

In [22]:
df_movie[(df_movie["budget"] == '/ff9qCepilowshEtG2GYWwzt2bs4.jpg')]#todrop 
df_movie[(df_movie["budget"] == '/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg')]
df_movie[(df_movie["budget"] == '/zaSf5OG7V8X8gqFvly88zDdRm46.jpg')]

,belongs_to_collection,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,genresid,genresname,production_companiesname,production_companiesid,production_countriesiso_3166_1,production_countriesname,spoken_languagesiso_639_1,spoken_languagesname,release_year
365441,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,2014-01-01,82.0,Released,Beware Of Frost Bites,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,17161.0,Odyssey Media,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
365442,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,2014-01-01,82.0,Released,Beware Of Frost Bites,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,18012.0,Pulser Productions,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
365443,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,2014-01-01,82.0,Released,Beware Of Frost Bites,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,18013.0,Rogue State,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
365444,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,2014-01-01,82.0,Released,Beware Of Frost Bites,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,23822.0,The Cartel,NaN,NaN,NaN,NaN,NaN,NaN,<NA>


In [23]:
df_movie = df_movie.loc[~df_movie['budget'].isin(['/ff9qCepilowshEtG2GYWwzt2bs4.jpg', '/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg', '/zaSf5OG7V8X8gqFvly88zDdRm46.jpg'])]





### Se extrae el valor 'name' de belongs_to_colections, que es una lista de diccionario a través de una función.

In [24]:

def extract_collection_name(collection_str):#Es una Función creada para extraer el valor de 'name' de un diccionario en formato de string
    if pd.isna(collection_str):
        return None
    try:
        collection_dict = ast.literal_eval(collection_str)
        return collection_dict.get('name', None)
    except (ValueError, SyntaxError):
        return None

# Creamos la nueva columna 'collection_name'con los datos extraidos de la función
df_movie['collection_name'] = df_movie['belongs_to_collection'].apply(extract_collection_name)

### REAGRUPAMIENTO DE DATOS: Aunque se están rompiendo algunos conceptos de normalización de datos en su primera y segunda forma, se ha decidido hacerlo con el objetivo de eliminar la duplicidad y reducir la cantidad de registros. Para ello, se realiza un group by con datos descriptivos, tales como los lenguajes de las películas y sus códigos ISO, así como el ID del género y el nombre del mismo

In [25]:
df_movie3 = df_movie.dropna(subset=['id']) #se crea un df nuevo para eliminar los id nulos

# Agrupar por el identificador único de la película y agregar listas de idiomas y géneros sin duplicados
df_movie_agg = df_movie3.groupby('id').agg({
    'spoken_languagesname': lambda x: ', '.join(set(i for i in x if pd.notna(i))),  # Filtra nulos, elimina duplicados y une con "," los valores de spoken_languagesname
    'spoken_languagesiso_639_1': lambda x: ', '.join(set(i for i in x if pd.notna(i))),  # Filtra nulos, elimina duplicados y une con "," los valores de spoken_languagesname_639
    'genresid': lambda x: ', '.join(map(str, set(int(i) for i in x if pd.notna(i)))),  # Convierte a enteros, elimina duplicados y une los ID con coma
    'genresname': lambda x: ', '.join(set(i for i in x if pd.notna(i)))  # Eliminamos duplicados y unimos con coma
}).reset_index()

df_movie_agg.columns = ['id', 'spoken_languages_names', 'spoken_languages_iso', 'genres_ids', 'genres_names']

# Unimos los datos agregados de nuevo con el DataFrame 
df_movie2 = df_movie3.merge(df_movie_agg, on='id', how='left')

In [26]:

columnas_a_eliminar = ['spoken_languagesname', 'spoken_languagesiso_639_1', 'genresid', 'genresname', 'belongs_to_collection']
df_movie2 = df_movie2.drop(columns=columnas_a_eliminar)

In [27]:
# Eliminamos duplicados de df_movie2 basados en la columna 'id', y volvemos a llamar df_movie a nuestro dataset
df_movie = df_movie2.drop_duplicates(subset=['id'])

### Cambiamos el tipo de dato de 'id' y 'budget'y comprobamos que funcione

In [28]:
df_movie

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,production_companiesname,production_companiesid,production_countriesiso_3166_1,production_countriesname,release_year,collection_name,spoken_languages_names,spoken_languages_iso,genres_ids,genres_names
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Pixar Animation Studios,3.0,US,United States of America,1995,Toy Story Collection,English,en,"16, 35, 10751","Family, Comedy, Animation"
3,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,TriStar Pictures,559.0,US,United States of America,1995,None,"English, Français","fr, en","12, 14, 10751","Family, Fantasy, Adventure"
21,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Warner Bros.,6194.0,US,United States of America,1995,Grumpy Old Men Collection,English,en,"35, 10749","Comedy, Romance"
25,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,Twentieth Century Fox Film Corporation,306.0,US,United States of America,1995,None,English,en,"18, 35, 10749","Comedy, Romance, Drama"
28,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Sandollar Productions,5842.0,US,United States of America,1995,Father of the Bride Collection,English,en,35,Comedy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418826,0,30840,en,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0.0,104.0,Released,NaN,Robin Hood,5.7,26.0,Westdeutscher Rundfunk (WDR),7025.0,CA,Canada,1991,None,English,en,"18, 28, 10749","Romance, Drama, Action"
418874,0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,Released,NaN,Century of Birthing,9.0,3.0,Sine Olivia,19653.0,PH,Philippines,2011,None,,tl,18,Drama
418875,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,A deadly game of wits.,Betrayal,3.8,6.0,American World Pictures,6165.0,US,United States of America,2003,None,English,en,"18, 28, 53","Thriller, Drama, Action"
418878,0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,Released,NaN,Satan Triumphant,0.0,0.0,Yermoliev,88753.0,RU,Russia,1917,None,,,,


In [29]:
df_movie.loc[:, 'id'] = df_movie['id'].astype(int)
df_movie['id'] = df_movie['id'].astype(int)
df_movie['production_companiesid'] = df_movie['production_companiesid'].replace([np.inf, -np.inf, np.nan], 0.0)
df_movie['production_companiesid'] = df_movie['production_companiesid'].astype(int)
df_movie['budget'] = df_movie['budget'].astype(float)

df_movie.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45346 entries, 0 to 418879
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   budget                          45346 non-null  float64       
 1   id                              45346 non-null  int32         
 2   original_language               45335 non-null  object        
 3   overview                        44405 non-null  object        
 4   popularity                      45346 non-null  object        
 5   release_date                    45346 non-null  datetime64[ns]
 6   revenue                         45346 non-null  float64       
 7   runtime                         45100 non-null  float64       
 8   status                          45266 non-null  object        
 9   tagline                         20387 non-null  object        
 10  title                           45346 non-null  object        
 11  vote_a

C:\Users\Giova\AppData\Local\Temp\ipykernel_9020\3384425545.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movie['id'] = df_movie['id'].astype(int)
C:\Users\Giova\AppData\Local\Temp\ipykernel_9020\3384425545.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movie['production_companiesid'] = df_movie['production_companiesid'].replace([np.inf, -np.inf, np.nan], 0.0)
C:\Users\Giova\AppData\Local\Temp\ipykernel_9020\3384425545.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

### Creamos la columna retorno, diviendo 'revenue' / 'budget'

In [30]:
import numpy as np

# Realizamos la división y asignas a una nueva columna 'return'
df_movie['return'] = df_movie['revenue'] / df_movie['budget']

# Reemplazamoss los valores nulos e infinitos con 0.0
df_movie['return'] = df_movie['return'].replace([np.inf, -np.inf, np.nan], 0.0)
df_movie['id'] = df_movie['id'].replace([np.inf, -np.inf, np.nan], 0.0)
df_movie

C:\Users\Giova\AppData\Local\Temp\ipykernel_9020\2455304294.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movie['return'] = df_movie['revenue'] / df_movie['budget']
C:\Users\Giova\AppData\Local\Temp\ipykernel_9020\2455304294.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movie['return'] = df_movie['return'].replace([np.inf, -np.inf, np.nan], 0.0)
C:\Users\Giova\AppData\Local\Temp\ipykernel_9020\2455304294.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,production_companiesname,production_companiesid,production_countriesiso_3166_1,production_countriesname,release_year,collection_name,spoken_languages_names,spoken_languages_iso,genres_ids,genres_names,return
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Pixar Animation Studios,3,US,United States of America,1995,Toy Story Collection,English,en,"16, 35, 10751","Family, Comedy, Animation",12.451801
3,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,TriStar Pictures,559,US,United States of America,1995,None,"English, Français","fr, en","12, 14, 10751","Family, Fantasy, Adventure",4.043035
21,0.0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Warner Bros.,6194,US,United States of America,1995,Grumpy Old Men Collection,English,en,"35, 10749","Comedy, Romance",0.000000
25,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,Twentieth Century Fox Film Corporation,306,US,United States of America,1995,None,English,en,"18, 35, 10749","Comedy, Romance, Drama",5.090760
28,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Sandollar Productions,5842,US,United States of America,1995,Father of the Bride Collection,English,en,35,Comedy,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418826,0.0,30840,en,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0.0,104.0,Released,NaN,Robin Hood,5.7,26.0,Westdeutscher Rundfunk (WDR),7025,CA,Canada,1991,None,English,en,"18, 28, 10749","Romance, Drama, Action",0.000000
418874,0.0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,Released,NaN,Century of Birthing,9.0,3.0,Sine Olivia,19653,PH,Philippines,2011,None,,tl,18,Drama,0.000000
418875,0.0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,A deadly game of wits.,Betrayal,3.8,6.0,American World Pictures,6165,US,United States of America,2003,None,English,en,"18, 28, 53","Thriller, Drama, Action",0.000000
418878,0.0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,Released,NaN,Satan Triumphant,0.0,0.0,Yermoliev,88753,RU,Russia,1917,None,,,,,0.000000


In [31]:
df_movie = df_movie[[
    'id', 'title', 'original_language', 'overview', 'tagline', 'status',
    'production_companiesname', 'production_companiesid', 
    'production_countriesiso_3166_1', 'production_countriesname', 
    'genres_names', 'genres_ids', 'spoken_languages_names', 
    'spoken_languages_iso', 'release_date', 'release_year', 'runtime', 
    'collection_name', 'popularity', 'vote_average', 'vote_count', 
    'budget', 'revenue', 'return'
]]




In [32]:
valores_unicos_status = df_movie['status'].unique()
print(valores_unicos_status)


['Released' nan 'Rumored' 'Post Production' 'In Production' 'Planned'
 'Canceled']


In [33]:

num_registros_originales = len(df_movie)

# Filtramos el DataFrame para mantener solo las películas en estado 'RELEASED'
df_movie_released = df_movie[df_movie['status'] == 'Released']

num_registros_eliminados = num_registros_originales - len(df_movie_released)

print(f"Cantidad de registros eliminados: {num_registros_eliminados}")

# Reasignar el DataFrame filtrado al original si lo deseas
df_movie = df_movie_released


Cantidad de registros eliminados: 439


In [34]:
df_movie.shape

(44907, 24)

In [35]:
df_movie.to_csv('../data/df_movie.csv', index=False)